In [1]:
import boto3

s3     = boto3.resource('s3')
bucket = s3.Bucket('dev-edl-dsc-data')

prefix ='TOC/validation/metadata/'   #  This is correct for your task

for k, obj in enumerate(bucket.objects.filter(Prefix=prefix)):
    print(obj.key, end="\n")  # This prints each file
    pass

print("N files in {}: {}".format(prefix, k))


TOC/validation/metadata/
TOC/validation/metadata/2024 01 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 02 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 03 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 04 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 05 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 06 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 07 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 08 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 09 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 10 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 11 Case PDF files Attachments Report.csv
TOC/validation/metadata/2024 12 Case PDF files Attachments Report.csv
TOC/validation/metadata/202401CasePDFfilesAttachmentsReport_s3locations.txt
TOC/validation/metadata/2024_CasePDFfilesAttachmentsReport_

In [2]:
s3_client = boto3.client('s3')

bucket = 'dev-edl-dsc-data'  # ✅ correct bucket
key = 'TOC/validation/metadata/toc_outcomes_2023_24_mrn_fin_caseid_merged.csv'
local_path = 'toc_outcomes_2023_24_mrn_fin_caseid_merged.csv'

s3_client.download_file(bucket, key, local_path)
print("Downloaded {} to {}".format(key, local_path))

Downloaded TOC/validation/metadata/toc_outcomes_2023_24_mrn_fin_caseid_merged.csv to toc_outcomes_2023_24_mrn_fin_caseid_merged.csv


In [3]:
import sagemaker
role = sagemaker.get_execution_role()
print(role)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::248877539307:role/DS-ExecutionRole


In [4]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        1.9G     0  1.9G   0% /dev
tmpfs           1.9G     0  1.9G   0% /dev/shm
tmpfs           1.9G  648K  1.9G   1% /run
tmpfs           1.9G     0  1.9G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G   79G   57G  59% /
/dev/nvme1n1    4.8G   27M  4.5G   1% /home/ec2-user/SageMaker
tmpfs           386M     0  386M   0% /run/user/1001
tmpfs           386M     0  386M   0% /run/user/1002
tmpfs           386M     0  386M   0% /run/user/1000
tmpfs           386M     0  386M   0% /run/user/0


In [5]:
bucket_name = 'dev-edl-dsc-data'
prefix = 'TOC_summarization/compiled_intake/'

s3 = boto3.client('s3')

total_size_bytes = 0
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
    for obj in page.get('Contents', []):
        total_size_bytes += obj['Size']

total_size_gb = total_size_bytes / (1024**3)
print(f"Total size in S3: {total_size_gb:.2f} GB")

Total size in S3: 0.27 GB


In [7]:
import boto3
import os
from datetime import datetime

def process_txt_files(bucket_name, folder_path, local_dir=None, file_type='.txt', mode='count'):
    """
    Lists or downloads .txt files from a given S3 location.
    
    mode = 'count'  -> only counts files
    mode = 'download' -> downloads files to local_dir
    """
    s3_client = boto3.client('s3')
    
    if local_dir and not os.path.exists(local_dir):
        os.makedirs(local_dir)

    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_path)
    
    file_count = 0
    for page in pages:
        for obj in page.get('Contents', []):
            if obj['Key'].endswith(file_type):
                file_count += 1
                file_name = os.path.basename(obj['Key'])
                local_file_path = os.path.join(local_dir if local_dir else '.', file_name)
                
                if mode == 'download':
                    # Uncomment to enable downloading
                    s3_client.download_file(bucket_name, obj['Key'], local_file_path)
                    pass
    
    print(f"Total {file_type} files in {folder_path}: {file_count}")


# === Run for COUNT only ===
bucket_name = 'dev-edl-dsc-data'
folder_path = 'TOC_summarization/compiled_intake/'
local_folder = 'compiled_intake_txt_files'  # folder for download later

start_time = datetime.now()
process_txt_files(bucket_name, folder_path, local_folder, '.txt', mode='download')
end_time = datetime.now()

print(f"Elapsed time: {end_time - start_time}")

Total .txt files in TOC_summarization/compiled_intake/: 11834
Elapsed time: 0:22:31.484603
